In [1]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras.applications import EfficientNetV2S, DenseNet201
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, concatenate, Input, BatchNormalization
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# from tensorflow.keras.optimizers import Adam
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import LabelEncoder
# import cv2
# import albumentations as A

# # Load and preprocess data
# train_df = pd.read_csv('/kaggle/input/aptos2019-blindness-detection/train.csv')
# train_df['diagnosis'] = train_df['diagnosis'].astype(str)
# train_df['id_code'] = train_df['id_code'].apply(lambda x: f"{x}.png")

# # Label encoding
# le = LabelEncoder()
# train_df['diagnosis'] = le.fit_transform(train_df['diagnosis'])

# def preprocess_image(image_path, target_size=(380, 380)):
#     img = cv2.imread(image_path)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
#     # Apply CLAHE
#     lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
#     l, a, b = cv2.split(lab)
#     clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
#     cl = clahe.apply(l)
#     limg = cv2.merge((cl,a,b))
#     img = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
    
#     # Resize
#     img = cv2.resize(img, target_size)
    
#     return img  # Return uint8 image

# # Augmentation pipeline
# aug_pipeline = A.Compose([
#     A.RandomRotate90(),
#     A.Flip(),
#     A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45),
#     A.OneOf([
#         A.OpticalDistortion(p=0.3),
#         A.GridDistortion(p=0.1),
#         A.ElasticTransform(p=0.3),
#     ], p=0.2),
#     A.OneOf([
#         A.GaussNoise(),
#         A.ISONoise(),
#         A.MultiplicativeNoise(),
#     ], p=0.2),
#     A.OneOf([
#         A.MotionBlur(p=0.2),
#         A.MedianBlur(blur_limit=3, p=0.1),
#         A.Blur(blur_limit=3, p=0.1),
#     ], p=0.2),
#     A.OneOf([
#         A.Sharpen(),
#         A.Emboss(),
#         A.RandomBrightnessContrast(),
#     ], p=0.3),
#     A.HueSaturationValue(p=0.3),
#     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# def custom_generator(dataframe, batch_size, augment=True, shuffle=True):
#     while True:
#         if shuffle:
#             dataframe = dataframe.sample(frac=1).reset_index(drop=True)
        
#         for start in range(0, len(dataframe), batch_size):
#             end = min(start + batch_size, len(dataframe))
#             batch_df = dataframe.iloc[start:end]
            
#             batch_images = []
#             batch_labels = []
            
#             for _, row in batch_df.iterrows():
#                 img_path = f"/kaggle/input/aptos2019-blindness-detection/train_images/{row['id_code']}"
#                 img = preprocess_image(img_path)
                
#                 if augment:
#                     augmented = aug_pipeline(image=img)
#                     img = augmented['image']
#                 else:
#                     img = aug_pipeline(image=img)['image']  # Only normalize
                
#                 batch_images.append(img)
#                 batch_labels.append(row['diagnosis'])
            
#             yield np.array(batch_images), np.array(batch_labels)

# # Create model
# def create_model(input_shape=(380, 380, 3), num_classes=5):
#     input_tensor = Input(shape=input_shape)
    
#     # EfficientNetV2S
#     base_model_1 = EfficientNetV2S(weights='imagenet', include_top=False, input_tensor=input_tensor)
#     x1 = base_model_1.output
#     x1 = GlobalAveragePooling2D()(x1)
#     x1 = BatchNormalization()(x1)
#     x1 = Dropout(0.5)(x1)
#     x1 = Dense(512, activation='relu')(x1)
#     x1 = BatchNormalization()(x1)
#     x1 = Dropout(0.3)(x1)
    
#     # DenseNet201
#     base_model_2 = DenseNet201(weights='imagenet', include_top=False, input_tensor=input_tensor)
#     x2 = base_model_2.output
#     x2 = GlobalAveragePooling2D()(x2)
#     x2 = BatchNormalization()(x2)
#     x2 = Dropout(0.5)(x2)
#     x2 = Dense(512, activation='relu')(x2)
#     x2 = BatchNormalization()(x2)
#     x2 = Dropout(0.3)(x2)
    
#     # Combine outputs
#     combined = concatenate([x1, x2])
#     combined = Dense(256, activation='relu')(combined)
#     combined = BatchNormalization()(combined)
#     combined = Dropout(0.3)(combined)
#     output = Dense(num_classes, activation='softmax')(combined)
    
#     model = Model(inputs=input_tensor, outputs=output)
#     return model

# # Implement k-fold cross-validation
# n_splits = 2
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['diagnosis']), 1):
#     print(f"Training Fold {fold}")
    
#     train_fold = train_df.iloc[train_idx]
#     val_fold = train_df.iloc[val_idx]
    
#     model = create_model()
    
#     # Compile model
#     optimizer = Adam(learning_rate=0.0001)
#     model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
#     # Callbacks
#     callbacks = [
#         EarlyStopping(patience=10, restore_best_weights=True),
#         ModelCheckpoint(f'best_model_fold_{fold}.keras', save_best_only=True),
#         ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-6)
#     ]
    
#     # Train model
#     batch_size = 16
#     train_generator = custom_generator(train_fold, batch_size)
#     val_generator = custom_generator(val_fold, batch_size, augment=False, shuffle=False)
    
#     history = model.fit(
#         train_generator,
#         steps_per_epoch=len(train_fold) // batch_size,
#         validation_data=val_generator,
#         validation_steps=len(val_fold) // batch_size,
#         epochs=100,
#         callbacks=callbacks,
#         verbose=2
#     )
    
#     # Print fold results
#     print(f"Fold {fold} - Best validation accuracy: {max(history.history['val_accuracy'])}")

# # After all folds, you can ensemble the models for final predictions
# print("Training completed for all folds.")

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2S, DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, concatenate, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import os

# Use GPU if available, otherwise use CPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Using GPU")
    except RuntimeError as e:
        print(e)
else:
    print("Using CPU")

# Load and preprocess data
train_df = pd.read_csv('/kaggle/input/aptos2019-blindness-detection/train.csv')
train_df['diagnosis'] = train_df['diagnosis'].astype(str)
train_df['id_code'] = train_df['id_code'].apply(lambda x: f"{x}.png")

# Label encoding
le = LabelEncoder()
train_df['diagnosis'] = le.fit_transform(train_df['diagnosis'])

def preprocess_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, (380, 380))
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    image = tf.image.random_hue(image, max_delta=0.2)
    return image, label

def create_dataset(dataframe, batch_size, is_training=True):
    image_paths = tf.constant([f"/kaggle/input/aptos2019-blindness-detection/train_images/{filename}" for filename in dataframe['id_code']])
    labels = tf.constant(dataframe['diagnosis'].values, dtype=tf.int32)
    
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    
    if is_training:
        dataset = dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

def create_model(input_shape=(380, 380, 3), num_classes=5):
    input_tensor = Input(shape=input_shape)
    
    base_model_1 = EfficientNetV2S(weights='imagenet', include_top=False, input_tensor=input_tensor)
    x1 = GlobalAveragePooling2D()(base_model_1.output)
    x1 = BatchNormalization()(x1)
    x1 = Dropout(0.5)(x1)
    x1 = Dense(512, activation='relu')(x1)
    x1 = BatchNormalization()(x1)
    x1 = Dropout(0.3)(x1)
    
    base_model_2 = DenseNet201(weights='imagenet', include_top=False, input_tensor=input_tensor)
    x2 = GlobalAveragePooling2D()(base_model_2.output)
    x2 = BatchNormalization()(x2)
    x2 = Dropout(0.5)(x2)
    x2 = Dense(512, activation='relu')(x2)
    x2 = BatchNormalization()(x2)
    x2 = Dropout(0.3)(x2)
    
    combined = concatenate([x1, x2])
    combined = Dense(256, activation='relu')(combined)
    combined = BatchNormalization()(combined)
    combined = Dropout(0.3)(combined)
    output = Dense(num_classes, activation='softmax')(combined)
    
    model = Model(inputs=input_tensor, outputs=output)
    return model

# Implement k-fold cross-validation
n_splits = 2
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['diagnosis']), 1):
    print(f"Training Fold {fold}")
    
    train_fold = train_df.iloc[train_idx].reset_index(drop=True)
    val_fold = train_df.iloc[val_idx].reset_index(drop=True)
    
    model = create_model()
    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Callbacks
    callbacks = [
        EarlyStopping(patience=15, restore_best_weights=True),
        ModelCheckpoint(f'best_model_fold_{fold}.keras', save_best_only=True),
        ReduceLROnPlateau(factor=0.5, patience=7, min_lr=1e-6)
    ]
    
    # Create datasets
    batch_size = 16  # Adjust based on your GPU memory
    train_dataset = create_dataset(train_fold, batch_size)
    val_dataset = create_dataset(val_fold, batch_size, is_training=False)
    
    # Train model
    try:
        history = model.fit(
            train_dataset,
            validation_data=val_dataset,
            epochs=20,
            callbacks=callbacks,
            steps_per_epoch=len(train_fold) // batch_size,
            validation_steps=len(val_fold) // batch_size,
            verbose=2
        )
        
        # Print fold results
        print(f"Fold {fold} - Best validation accuracy: {max(history.history['val_accuracy'])}")
    except Exception as e:
        print(f"An error occurred during training fold {fold}: {str(e)}")
        continue

print("Training completed for all folds.")

Using GPU
Training Fold 1
82420632/82420632 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/20


I0000 00:00:1725685890.587673      65 service.cc:145] XLA service 0x7c2148005110 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725685890.587727      65 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2024-09-07 05:20:41.539545: E external/local_xla/xla/service/slow_operation_alarm.cc:65] 
********************************
[Compiling module a_inference_one_step_on_data_235599__.163720] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
I0000 00:00:1725686459.521716      65 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_210', 8 bytes spill stores, 68 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_14', 28 bytes spill stores, 28 bytes spill loads
ptxas warning : Registers 

114/114 - 986s - 9s/step - accuracy: 0.5027 - loss: 1.5110 - val_accuracy: 0.6168 - val_loss: 0.9553 - learning_rate: 1.0000e-04
Epoch 2/20


I0000 00:00:1725687213.560455      65 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_1512', 344 bytes spill stores, 316 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_1511', 192 bytes spill stores, 192 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_1510', 292 bytes spill stores, 292 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_1509', 300 bytes spill stores, 300 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_1508', 308 bytes spill stores, 308 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_1507', 316 bytes spill stores, 316 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_1505', 8 bytes

114/114 - 607s - 5s/step - accuracy: 0.7143 - loss: 1.0946 - val_accuracy: 0.5714 - val_loss: 0.8386 - learning_rate: 1.0000e-04
Epoch 3/20
114/114 - 196s - 2s/step - accuracy: 0.6338 - loss: 1.1382 - val_accuracy: 0.7434 - val_loss: 0.7204 - learning_rate: 1.0000e-04
Epoch 4/20
114/114 - 6s - 52ms/step - accuracy: 0.8571 - loss: 0.6524 - val_accuracy: 0.8571 - val_loss: 0.4012 - learning_rate: 1.0000e-04
Epoch 5/20
114/114 - 201s - 2s/step - accuracy: 0.6771 - loss: 1.0291 - val_accuracy: 0.6721 - val_loss: 0.8056 - learning_rate: 1.0000e-04
Epoch 6/20
114/114 - 1s - 5ms/step - accuracy: 0.8571 - loss: 0.2888 - val_accuracy: 0.8571 - val_loss: 1.0308 - learning_rate: 1.0000e-04
Epoch 7/20
114/114 - 188s - 2s/step - accuracy: 0.6870 - loss: 0.9418 - val_accuracy: 0.7615 - val_loss: 0.6568 - learning_rate: 1.0000e-04
Epoch 8/20
114/114 - 1s - 5ms/step - accuracy: 0.7143 - loss: 1.5937 - val_accuracy: 0.8571 - val_loss: 0.4995 - learning_rate: 1.0000e-04
Epoch 9/20
114/114 - 187s - 2s/st